## USE THE NEW CSV FROM DRIVE

In [174]:
import pandas as pd
import spacy
import re
from collections import defaultdict
from nltk.stem import WordNetLemmatizer

In [132]:
data = pd.read_csv("edited.csv")

In [133]:
documents = data["title"].tolist()
score = data ["score"].tolist()
subreddit = data["subreddit"].tolist()
permalink = data["permalink"].tolist()

In [134]:
#build inverted index for subreddits
inverted_idx = defaultdict(list)
for d in range(len(subreddit)):
    #print (type(subreddit[d]))
    inverted_idx[subreddit[d]].append(d)

In [175]:
lemmatizer = WordNetLemmatizer()

In [135]:
a = data["content_categories"].dropna()

In [136]:
categories = defaultdict(list)
for idx, lst in a.iteritems():
    c = lst.replace('[', "").replace("]", "").replace('\'', "").replace(" ", "")
    l = c.split(",")
    for i in l:
        categories[i].append(idx)

In [176]:
init = list(categories.keys())
topics = {i:set() for i in init}
for key in categories:
    for idx in categories[key]:
        topics[key].add(data.iloc[idx]["subreddit"])
    topics[key].add(lemmatizer.lemmatize(key))

In [177]:
topics

{'funny': {'funny', 'lol'},
 'gaming': {'2007scape',
  'AFL',
  'Battlecars',
  'CODZombies',
  'ClashRoyale',
  'DBZDokkanBattle',
  'DarkSouls2',
  'DestinyTheGame',
  'Diablo',
  'DnD',
  'DotA2',
  'EndlessFrontier',
  'Eve',
  'FifaVideos',
  'FireEmblemHeroes',
  'FryeMadden',
  'FryeMadden2',
  'FuturamaWOTgame',
  'GamePhysics',
  'GamerVideos',
  'GlobalOffensive',
  'Guildwars2',
  'Infinitewarfare',
  'MaddenCC',
  'MaddenUltimateTeam',
  'Minecraft',
  'NintendoSwitch',
  'NoMansSkyTheGame',
  'OpTicGaming',
  'Overwatch',
  'PS4',
  'PUBATTLEGROUNDS',
  'Planetside',
  'Rockband',
  'RocketLeague',
  'Smite',
  'StarWarsBattlefront',
  'Steam',
  'TeamSolomid',
  'Terraria',
  'WorldOfWarships',
  'Yogscast',
  'bloodborne',
  'boardgames',
  'csgojerk',
  'darksouls3',
  'dragonquest',
  'fnafcringe',
  'gaming',
  'gamingvids',
  'gtaonline',
  'hediyhstone',
  'incremental_games',
  'leagueoflegends',
  'magicTCG',
  'nintendo',
  'osugame',
  'pm_your_pokemon_team',
  

In [178]:
sub_cat = {}
for key in topics:
    for i in topics[key]:
        sub_cat[i] = key
print (sub_cat)

{'lol': 'funny', 'funny': 'funny', 'NintendoSwitch': 'gaming', 'gaming': 'gaming', 'unturned': 'gaming', 'GlobalOffensive': 'gaming', 'splatoon': 'gaming', 'Yogscast': 'gaming', 'Battlecars': 'gaming', 'Infinitewarfare': 'gaming', 'dragonquest': 'gaming', 'PUBATTLEGROUNDS': 'gaming', 'DnD': 'gaming', 'runescape': 'gaming', 'FifaVideos': 'gaming', 'starcraft': 'gaming', '2007scape': 'gaming', 'Guildwars2': 'gaming', 'skyrim': 'gaming', 'titanfall': 'gaming', 'DBZDokkanBattle': 'gaming', 'twitchplayspokemon': 'gaming', 'fnafcringe': 'gaming', 'gamingvids': 'gaming', 'MaddenUltimateTeam': 'gaming', 'GamerVideos': 'gaming', 'WorldOfWarships': 'gaming', 'Eve': 'gaming', 'Steam': 'gaming', 'magicTCG': 'gaming', 'reddeadredemption': 'gaming', 'Diablo': 'gaming', 'Smite': 'gaming', 'worldbuilding': 'gaming', 'Minecraft': 'gaming', 'ClashRoyale': 'gaming', 'boardgames': 'gaming', 'Overwatch': 'gaming', 'pm_your_pokemon_team': 'gaming', 'CODZombies': 'gaming', 'NoMansSkyTheGame': 'gaming', 'smas

## If we return no results from above... try below

query = "pig"
category = "animals"
we run the query search thing, and there's a subreddit for pig (but it's not one of the ones I manually labeled) so it doesn't appear in the dictonary above. So I take the subreddits from the returned results from simply looking up the query and run synayom search to see if I can find a subcategory that fits. 


replace most_similar with word embeddings

In [179]:
import spacy
nlp = spacy.load('en_core_web_lg')
vocab = [w for w in nlp.vocab if w.is_lower and w.prob >= -15 and w.vector.any()]
def most_similar(word, top=20):
    by_similarity = sorted(vocab, key=lambda w: word.similarity(w), reverse=True)
    return by_similarity[:top]

In [180]:
subreddit = "pig"
def unknown_subreddit(subreddit, sub_cat):
    if subreddit not in sub_cat:
        syn = [w.lower_ for w in most_similar(nlp.vocab[subreddit])]
        for word in syn:
            if word in sub_cat:
                sub_cat[subreddit] = sub_cat[word]
                return sub_cat[word]
        return None
    else:
        return sub_cat[subreddit]

In [181]:
print (unknown_subreddit("pig", sub_cat))

animals


In [182]:
[w.lower_ for w in most_similar(nlp.vocab[subreddit])]

['pig',
 'pigs',
 'cow',
 'rabbit',
 'hog',
 'guinea',
 'piglet',
 'sheep',
 'goat',
 'chickens',
 'monkey',
 'animal',
 'rat',
 'piggy',
 'chicken',
 'pork',
 'meat',
 'duck',
 'swine',
 'donkey']